<a href="https://colab.research.google.com/github/deekshi07/first_project/blob/master/AI47Labs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""Untitled11.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1aX2GE4kNMvpbFrGf47MxrYxv7UZgCSAq
"""

!pip install requests beautifulsoup4 pandas

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List of top 50 hospital URLs
hospital_urls = [
    "https://www.mayoclinic.org",
    "https://my.clevelandclinic.org",
    "https://www.massgeneral.org",
    "https://www.hopkinsmedicine.org",
    "https://www.charite.de/en",
    "https://www.uhn.ca",
    "https://www.mountsinai.org",
    "https://www.uclahealth.org",
    "https://www.karolinska.se/en",
    "https://www.sgh.com.sg",
    "https://www.shebaonline.org",
    "https://www.nyp.org",
    "https://www.uofmhealth.org",
    "https://www.thermh.org.au",
    "https://stanfordhealthcare.org",
    "https://www.mountelizabeth.com.sg",
    "https://www.nuh.com.sg",
    "https://www.oslouniversitetssykehus.no",
    "https://www.kch.nhs.uk",
    "https://www.guysandstthomas.nhs.uk",
    "https://www.aiims.edu",
    "https://www.madrid.org",
    "https://www.samsunghospital.com",
    "https://www.usz.ch",
    "https://www.pumch.cn",
    "https://www.snuh.org",
    "https://www.einstein.br",
    "https://www.mskcc.org",
    "https://www.aphp.fr",
    "https://www.clevelandclinicabudhabi.ae",
    "https://www.imperial.nhs.uk",
    "https://www.cmch-vellore.edu",
    "https://www.apollohospitals.com",
    "https://www.bumrungrad.com",
    "https://www.kfshrc.edu.sa",
    "https://www.uclh.nhs.uk",
    "https://www.stgeorges.nhs.uk",
    "https://www.snec.com.sg",
    "https://www.uza.be",
    "https://www.uhn.ca/TWH",
    "https://www.dukehealth.org",
    "https://www.cuh.nhs.uk",
    "https://www.ynhh.org"
]

# Function to scrape doctor data and other attributes from each hospital
def scrape_hospital_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    doctors = []

    # Scraping page title for validation
    page_title = soup.title.string.strip() if soup.title else "N/A"

    # Extracting hospital name
    hospital_name = soup.find('meta', attrs={'name': 'og:site_name'})  # Example of meta tag for hospital name
    hospital_name = hospital_name['content'] if hospital_name else "N/A"

    # Extracting departments or services offered
    services_section = soup.find('div', class_='services')  # Adjust the class based on actual HTML structure
    services = services_section.text.strip() if services_section else "N/A"

    # Extracting hospital location
    location_section = soup.find('div', class_='location')  # Adjust based on actual HTML structure
    hospital_location = location_section.text.strip() if location_section else "N/A"

    # Extracting doctor profiles
    doctor_profiles = soup.find_all('div', class_='doctor-profile')  # Adjust this class based on actual HTML structure

    if not doctor_profiles:
        doctors.append({
            'Hospital': hospital_name,
            'Page Title': page_title,
            'Doctor Name': "N/A",
            'Specialization': "N/A",
            'Departments/Services': services,
            'Location': hospital_location,
            'Photo': "N/A",
            'Contact Info': "N/A",
            'Education': "N/A",
            'Experience': "N/A"
        })
    else:
        for profile in doctor_profiles:
            name = profile.find('h2').text.strip() if profile.find('h2') else "N/A"
            specialization = profile.find('p', class_='specialization').text.strip() if profile.find('p', class_='specialization') else "N/A"

            # Extract additional attributes
            photo = profile.find('img')['src'] if profile.find('img') else "N/A"  # Adjust based on actual HTML structure
            contact_info = profile.find('p', class_='contact-info').text.strip() if profile.find('p', class_='contact-info') else "N/A"  # Adjust based on actual HTML structure
            education = profile.find('p', class_='education').text.strip() if profile.find('p', class_='education') else "N/A"  # Adjust based on actual HTML structure
            experience = profile.find('p', class_='experience').text.strip() if profile.find('p', class_='experience') else "N/A"  # Adjust based on actual HTML structure

            doctors.append({
                'Hospital': hospital_name,
                'Page Title': page_title,
                'Doctor Name': name,
                'Specialization': specialization,
                'Departments/Services': services,
                'Location': hospital_location,
                'Photo': photo,
                'Contact Info': contact_info,
                'Education': education,
                'Experience': experience
            })

    return doctors

# Scraping data for all hospitals
all_doctor_data = []

for hospital_url in hospital_urls:
    try:
        doctor_data = scrape_hospital_data(hospital_url)
        all_doctor_data.extend(doctor_data)
        print(f"Scraped data from {hospital_url}")
    except Exception as e:
        print(f"Failed to scrape {hospital_url}: {e}")

# Converting the scraped data into a DataFrame
df = pd.DataFrame(all_doctor_data)

# Displaying the data after scraping
print("Scraped Data:")
print(df.head())  # Display the first few rows of the DataFrame

# Saving the data to a CSV file
csv_path = 'hospital_doctor_data.csv'
df.to_csv(csv_path, index=False)

# Reading and displaying the contents of the saved CSV file
if os.path.exists(csv_path):
    with open(csv_path, 'r') as file:
        cleaned_content = file.read()
        print("\nContent of 'hospital_doctor_data.csv':")
        print(cleaned_content)
else:
    print(f"The file '{csv_path}' does not exist.")

import pandas as pd

# Load the scraped data from CSV
df = pd.read_csv('hospital_doctor_data.csv')

# Display the initial data
print("Initial Data:")
print(df.head())

# Remove duplicates
df.drop_duplicates(inplace=True)

# Handle missing values
# You can either drop rows with missing values or fill them in with a placeholder
df.fillna("Unknown", inplace=True)  # Filling missing values with "Unknown"

# Reset index after cleaning
df.reset_index(drop=True, inplace=True)

# Display cleaned data
print("Cleaned Data:")
print(df.head())

# Save the cleaned data to a new CSV file
cleaned_csv_path = 'cleaned_hospital_doctor_data.csv'
df.to_csv(cleaned_csv_path, index=False)

from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load your cleaned data
cleaned_data = pd.read_csv(cleaned_csv_path)

# Convert DataFrame to a list of strings for training
training_data = cleaned_data.apply(lambda row: f"Dr. {row['Doctor Name']} specializes in {row['Specialization']} at {row['Hospital']}. Services: {row['Departments/Services']}.", axis=1).tolist()

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the pad token to the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the data
inputs = tokenizer(training_data, return_tensors='pt', padding=True, truncation=True)

# Fine-tune the model
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

# Training loop
for epoch in range(3):  # Adjust number of epochs as needed
    outputs = model(**inputs, labels=inputs['input_ids'])
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f"Epoch {epoch + 1}: Loss = {loss.item()}")

# Save the fine-tuned model
model.save_pretrained('fine_tuned_gpt_model')
tokenizer.save_pretrained('fine_tuned_gpt_model')

from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained('fine_tuned_gpt_model')
tokenizer = GPT2Tokenizer.from_pretrained('fine_tuned_gpt_model')

# Function to generate text based on a prompt
def generate_text(prompt, max_length=100):
    inputs = tokenizer.encode(prompt, return_tensors='pt')  # Encode the prompt
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)  # Decode the generated text

# Example prompts
prompts = [
    "Tell me about Dr. Smith specializing in cardiology.",
    "What services does Mayo Clinic offer?",
    "Give me details about Dr. John Doe and his specialties."
]

# Generate text for each prompt
for prompt in prompts:
    print(f"Prompt: {prompt}")
    generated_text = generate_text(prompt)
    print(f"Generated Text: {generated_text}\n")

from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained('fine_tuned_gpt_model')
tokenizer = GPT2Tokenizer.from_pretrained('fine_tuned_gpt_model')

# Function to generate text based on a prompt
def generate_text(prompt, max_length=100):
    inputs = tokenizer.encode(prompt, return_tensors='pt')  # Encode the prompt
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)  # Decode the generated text

# Main loop to interact with the user
def main():
    while True:
        user_prompt = input("Enter your prompt (or type 'exit' to quit): ")
        if user_prompt.lower() == 'exit':
            print("Exiting the program.")
            break
        generated_text = generate_text(user_prompt)
        print(f"Generated Text: {generated_text}\n")

# Run the main function
if __name__ == "__main__":
    main()

Scraped data from https://www.mayoclinic.org
Scraped data from https://my.clevelandclinic.org
Scraped data from https://www.massgeneral.org
Scraped data from https://www.hopkinsmedicine.org
Scraped data from https://www.charite.de/en
Scraped data from https://www.uhn.ca
Scraped data from https://www.mountsinai.org
Scraped data from https://www.uclahealth.org
Scraped data from https://www.karolinska.se/en
Scraped data from https://www.sgh.com.sg
Scraped data from https://www.shebaonline.org
Scraped data from https://www.nyp.org
Scraped data from https://www.uofmhealth.org
Scraped data from https://www.thermh.org.au
Scraped data from https://stanfordhealthcare.org
Scraped data from https://www.mountelizabeth.com.sg
Scraped data from https://www.nuh.com.sg
Scraped data from https://www.oslouniversitetssykehus.no
Scraped data from https://www.kch.nhs.uk
Scraped data from https://www.guysandstthomas.nhs.uk
Scraped data from https://www.aiims.edu
Scraped data from https://www.madrid.org
Scra

<ipython-input-2-0140aaa87c77>:162: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("Unknown", inplace=True)  # Filling missing values with "Unknown"
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Generated Text: What services does Mayo Clinic offer?

The Mayo Clinics of North America offers a wide range of services to help you find the right provider for your needs.
...
,.,.,,,
 (. ).

Prompt: Give me details about Dr. John Doe and his specialties.
Generated Text: Give me details about Dr. John Doe and his specialties.

Dr. Doe: Dr John is a specialist in the field of the human body. He specializes in:
. Drs. D. and Ds., Dr D., and Dr S.D. are specialists in human anatomy. They specialize in Human Anatomy.


Doe: Doe is an expert in all aspects of human physiology. His specialty is:

. The Dr is the Dr, the Doctor

Enter your prompt (or type 'exit' to quit): Tell me about Dr Jhon


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: Tell me about Dr Jhon.

Dr Jon: I'm a doctor. I've been in the field for over 20 years. My specialty is in medicine. You can call me anytime. Dr. Joffe: Dr, I am DrJoffey. Thank you. [pause]
...
[pause]....
I'm Drjoff. Thanks for the call.



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: tell me about Dr smith.

Dr. Smith: I'm a doctor. I am a physician. Dr. Smith: You're a surgeon. You are a nurse. Doctor Smiths: Yes. Yes, I know. No, no, you're not. It's not like I can do that. But I do. And I have a lot of experience. So I understand. Thank you. [pause]
...
[pause]..."
"

